In [94]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np



In [95]:
#file_path = '/Users/anasoni/Downloads/Big_Data_DIS/Final_project/imdb_top_1000.csv'
file_path = 'data/imdb_top_1000.csv'

df_raw = pd.read_csv(file_path)


#df_raw['Runtime'] = df_raw['Runtime'].str.extract('(\d+)').astype(int)
df_raw['Genre'] = df_raw['Genre'].str.split(', ')

df_raw['Gross'] = df_raw['Gross'].str.replace(",", "").astype(float)
#df_raw['Released_Year'] = df_raw['Released_Year'].astype(float) # theres one pg so hard to compare here lol will look at this later

tfidf = TfidfVectorizer(stop_words='english')
df_raw['overview_tfidf'] = list(tfidf.fit_transform(df_raw['Overview']).toarray())

encoder = OneHotEncoder()
directors_encoded = encoder.fit_transform(df_raw[['Director']])

# Initialize MultiLabelBinarizer and transform genres
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(df_raw['Genre'])
df_raw['genre_mlb'] = list(mlb.fit_transform(df_raw['Genre']))


In [96]:
numeric_columns = ['Released_Year', 'IMDB_Rating', 'Meta_score']

# Convert selected columns to numeric type (if not already)
df_raw[numeric_columns] = df_raw[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Calculate the maximum and minimum values for each numeric column
max_values = df_raw[numeric_columns].max()
min_values = df_raw[numeric_columns].min()

# Display the maximum and minimum values for each column
print("Maximum values:")
print(max_values)
print("\nMinimum values:")
print(min_values)


Maximum values:
Released_Year    2020.0
IMDB_Rating         9.3
Meta_score        100.0
dtype: float64

Minimum values:
Released_Year    1920.0
IMDB_Rating         7.6
Meta_score         28.0
dtype: float64


In [97]:
# Define maximum and minimum values for numeric features
max_values = {
    'Released_Year': max_values['Released_Year'],
    'IMDB_Rating': max_values['IMDB_Rating'],
    'Meta_score': max_values['Meta_score']
}

min_values = {
    'Released_Year': min_values['Released_Year'],
    'IMDB_Rating': min_values['IMDB_Rating'],
    'Meta_score': min_values['Meta_score']
}

In [98]:
# Function to calculate similarity based on numeric difference and custom scaling factor (k)
def calculate_numeric_similarity(x, y, feature_name, k):
    max_val = max_values[feature_name]
    min_val = min_values[feature_name]
    range_val = max_val - min_val
    
    # Handle missing or invalid values
    if pd.isnull(x) or pd.isnull(y) or range_val == 0:
        return 0.0  # Return 0 similarity for missing or invalid values
    
    difference = abs(x - y)
    similarity = max(100 - k * (difference / range_val) * 100, 0)
    
    # Normalize similarity between 0 and 1
    similarity_normalized = similarity / 100.0
    return similarity_normalized

# Example usage with customizable scaling factors (k) for each numeric feature
def test_similarity():
    file_path = 'data/imdb_top_1000.csv'
    df_raw = pd.read_csv(file_path)

    # Convert selected columns to numeric types
    numeric_columns = ['Released_Year', 'IMDB_Rating', 'Meta_score']
    df_raw[numeric_columns] = df_raw[numeric_columns].apply(pd.to_numeric, errors='coerce')

    # Example movie indices
    movie1_index = 0
    movie2_index = 1

    # Select movies based on indices
    movie1 = df_raw.iloc[movie1_index]
    movie2 = df_raw.iloc[movie2_index]

    # Define scaling factors (k) for each numeric feature
    k_year = 0.7
    k_imdb = 0.4
    k_metascore = 0.6

    # Calculate similarity based on different numeric features with custom scaling factors (k)
    similarity_year = calculate_numeric_similarity(movie1['Released_Year'], movie2['Released_Year'], 'Released_Year', k_year)
    similarity_imdb = calculate_numeric_similarity(movie1['IMDB_Rating'], movie2['IMDB_Rating'], 'IMDB_Rating', k_imdb)
    similarity_metascore = calculate_numeric_similarity(movie1['Meta_score'], movie2['Meta_score'], 'Meta_score', k_metascore)

    # Print similarity scores as decimal values between 0 and 1
    print(f"Similarity (Released Year): {similarity_year:.4f}")
    print(f"Similarity (IMDB Rating): {similarity_imdb:.4f}")
    print(f"Similarity (Metascore): {similarity_metascore:.4f}")

# Call the function to test similarity calculation
test_similarity()

Similarity (Released Year): 0.8460
Similarity (IMDB Rating): 0.9765
Similarity (Metascore): 0.8333


In [99]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def filter_similarity(desc, genre, director, imdb, critic, year):
    description_sim = .35 * desc
    genre_sim = .25 * genre
    released_year = .05 * year
    imdb_rating = .1 * imdb
    meta_score = .2 * critic
    director_sim = .05 * director

    return (description_sim + genre_sim + released_year + imdb_rating + meta_score + director_sim) * 100

In [100]:
# Example function to calculate similarity
def calculate_similarity(df, index):
    similarities = {}
    target = df.iloc[index]
    for i, row in df.iterrows():
        if i != index:

            # This calculates description similarity
            description_similarity = 0
            # Combine weighted scores from different features
            description_similarity += cosine_similarity([target['overview_tfidf']], [row['overview_tfidf']])[0][0]
            # Add other similarities, weighted appropriately
            
            # This calculates genre similarities 
            intersection = np.logical_and(target['genre_mlb'], row["genre_mlb"])
            union = np.logical_or(target['genre_mlb'], row["genre_mlb"])
            genre_similarity = intersection.sum() / float(union.sum())

            # This calculates whether movies share a director or not
            director_similarity = 1 if target['Director'] == row['Director'] else 0
            
            # Define scaling factors (k) for each numeric feature
            k_year = 0.7
            k_imdb = 5
            k_metascore = 0.6

            # Release year
            year_similarity = calculate_numeric_similarity(target['Released_Year'], row['Released_Year'], 'Released_Year', k_year)
            imdb_similarity = calculate_numeric_similarity(target['IMDB_Rating'], row['IMDB_Rating'], 'IMDB_Rating', k_imdb)
            meta_similarity = calculate_numeric_similarity(target['Meta_score'], row['Meta_score'], 'Meta_score', k_metascore)
            

            similarities[row['Series_Title']] = filter_similarity(director_similarity,genre_similarity, director_similarity,imdb_similarity, meta_similarity, year_similarity)
    return sorted(similarities.items(), key=lambda x: x[1], reverse=True)

def recommend_movies(title, top_n=10):
    index = df_raw[df_raw['Series_Title'] == title].index[0]
    recommendations = calculate_similarity(df_raw, index)
    return recommendations[:top_n]



# def calculate_similarity(df, index):
    

In [101]:
recommend_movies('Coco')

[('Toy Story 3', 79.53931372549017),
 ('WALL·E', 57.35166666666667),
 ('Hauru no ugoku shiro', 53.49598039215684),
 ('Sen to Chihiro no kamikakushi', 51.057647058823555),
 ('The Secret of Kells', 49.720000000000006),
 ('Kari-gurashi no Arietti', 49.58833333333333),
 ('Mulan', 47.66833333333334),
 ('Rupan sansei: Kariosutoro no shiro', 47.00333333333334),
 ('Spider-Man: Into the Spider-Verse', 46.465),
 ('Mononoke-hime', 45.96666666666667)]